In [2]:
# from google.colab import userdata
# OPENAI_API_KEY=userdata.get('OPEN_AI_TOKEN')

In [ ]:
import os
OPENAI_API_KEY=os.environ['OPENAI_API_KEY' ]

In [ ]:
from openai import OpenAI # type: ignore

In [3]:
client = OpenAI()

In [9]:
client.files.create(file=open("drug_malady_data_transformed.jsonl","rb"),purpose='fine-tune')

FileObject(id='file-ILjgzvk3KTRZiiPL5mvVejsd', bytes=620084, created_at=1732072932, filename='drug_malady_data_transformed.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [10]:
file_id='file-ILjgzvk3KTRZiiPL5mvVejsd'

In [11]:
files = client.files.list()

In [18]:
for file in files.data:
  print(file)

FileObject(id='file-ILjgzvk3KTRZiiPL5mvVejsd', bytes=620084, created_at=1732072932, filename='drug_malady_data_transformed.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-Khry9hq2rnPnfwZytLeDxddj', bytes=2156, created_at=1732072666, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)
FileObject(id='file-pAe1fZPjdvWT2GXYaexEc01r', bytes=6429, created_at=1732072291, filename='test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-lJc8clBfKiKHoha38vCfPiQ3', bytes=2192, created_at=1732057951, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)
FileObject(id='file-6KrtwhpRz9y6rKV3vrY3wgJJ', bytes=6429, created_at=1732057342, filename='test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-iszdVT7YRAPrQGqZ

In [15]:
for file in files.data:
  if file.filename == 'drug_malady_data_transformed.jsonl':
    training_file_id = file.id

In [16]:
client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-8vvruUDyJhBK0ZyU1IWabTWG', created_at=1732073682, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-H5AqcfWE5doF86BXLEyduPAE', result_files=[], seed=142064352, status='validating_files', trained_tokens=None, training_file='file-ILjgzvk3KTRZiiPL5mvVejsd', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [23]:
jobs=client.fine_tuning.jobs.list()

In [ ]:
# Configure the model ID. Change this to your model ID.
drug_fine_tune_llm='ft:gpt-3.5-turbo-0125:college::AVWObtTY'

# Let's use a drug from each class
drugs = [
    "A CN Gel(Topical) 20gmA CN Soap 75gm",  # Class 0
    "Addnok Tablet 20'S",                    # Class 1
    "ABICET M Tablet 10's",                  # Class 2
]

# Returns a drug class for each drug
for drug_name in drugs:
    prompt = "Drug: {}\nMalady:".format(drug_name)
    print(prompt)
    response = client.chat.completions.create(
        model=drug_fine_tune_llm,
         messages=[{"role": "system", "content": "You are a medical assistant specializing in drug and malady classification. You help classify drugs and match them with possible ailments."}, 
              {"role": "user", "content": prompt}],
    )
    # Print the generated text
    drug_class = response.choices[0].message.content.strip()  # type: ignore
    # The result should be 0, 1, and 2
    print('Class:',drug_class)

Drug: A CN Gel(Topical) 20gmA CN Soap 75gm
Malady:
Class: 0
Drug: Addnok Tablet 20'S
Malady:
Class: 5
Drug: ABICET M Tablet 10's
Malady:
Class: 2
